## Assignment 4 - MDP

## Importing Libraries

In [2]:
import cs7641assn4 as a4
import numpy as np
import pandas as pd
import warnings
import xlrd

pd.set_option('display.max_columns', 35)
# pd.reset_option("display.max_columns")

## Importing Settings

In [36]:

# Import settings, uncomment to read settings from an excel spreadsheet
# Imports 'default_settings.xlsx' spreadsheet
settings = pd.read_excel('C:\\Users\\Mike\\Documents\\Georgia Tech\\Machine Learning\\hw4-ncook-solution-master\\default_settings.xlsx').to_dict()

# Determine the number of runs
n_settings = len(settings['rH'])

In [37]:
for n in range(n_settings):
    # Load settings
    rH = settings['rH'][n] # -1 #-5 # reward for H(ole)
    rG = settings['rG'][n] #1 # 10 # reward for G(oal)
    rF = settings['rF'][n] #-0.2# reward includes S(tart) and F(rozen)
    size = settings['size'][n] #4 # height and width of square gridworld, [4, 8, 16] are included in cs7641assn4.py 
    p = settings['p'][n] #0.8 # if generating a random map probability that a grid will be F(rozen)
    map_name = 'x'.join([str(size)]*2) # None, if you want a random map
    desc = a4.MAPS[map_name] # None, if you want a random map
    is_slippery = settings['is_slippery'][n] #False
    render_initial = settings['render_initial'][n] # True

    epsilon = settings['epsilon'][n] #1e-8 # convergence threshold for policy/value iteration
    gamma = settings['gamma'][n] #0.8 # discount parameter for past policy/value iterations
    max_iter = settings['max_iter'][n] #10000 # maximum iterations for slowly converging policy/value iteration 

    # Qlearning(env, rH=0, rG=1, rF=0, qepsilon=0.1, lr=0.8, gamma=0.95, episodes=10000)
    qepsilon = settings['qepsilon'][n] #0.1 # epsilon value for the Q-learning epsilon greedy strategy
    lr = settings['lr'][n] #0.8 # Q-learning rate
    qgamma = settings['qgamma'][n] #0.95 # Q-Learning discount factor
    episodes = settings['episodes'][n] #10000 # number of Q-learning episodes
    initial = settings['initial'][n] #0 # value to initialize the Q grid
    decay = settings['decay'][n] #True

    # Printing options
    report = settings['report'][n] #True # For cs7641assn4.py policy and value iteration functions
    display_print = settings['display_print'][n] #True # For this script

    # Create Environment
    env = a4.getEnv(env_id='hw4-FrozenLake-v0', rH=rH, rG=rG, rF=rF, 
                    desc=desc,  
                    is_slippery=is_slippery, render_initial=True)

    # Store a representation of the map
    env_desc = env.desc.astype('<U8')

    # Store a representation of the state rewards
    env_rs = a4.getStateReward(env)

    if display_print:
        # Display reward at each state
        print('\n--Reward Values at Each State--')
        a4.matprint(a4.print_value(env_rs, width=size, height=size))
        
    ## Policy Iteration
    print('\n--Policy Iteration TimeIt--')
    pi_time = %timeit -o a4.policy_iteration(env, epsilon, gamma, max_iter, report=False)
    
    pi_V, pi_policy, pi_epochs = a4.policy_iteration(env, epsilon, gamma, max_iter, report=report)

    pi_policy_arrows = a4.print_policy(pi_policy, width=size, height=size)

    if display_print:
        # Display values
        print('\n--Policy Iteration Values in grid order--')
        a4.matprint(a4.print_value(pi_V, width=size, height=size))

        # Display policy
        print('\n--Policy Iteration Policy Matrix--')
        a4.matprint(pi_policy_arrows)
        
    ## Value Iteration
    print('\n--Value Iteration TimeIt--')
    vi_time = %timeit -o a4.valueIteration(env, epsilon, gamma, max_iter, report=False)
    
    vi_V, vi_epochs = a4.valueIteration(env, epsilon, gamma, max_iter, report=report)

    vi_policy = a4.value_to_policy(env, V=vi_V, gamma=gamma)

    vi_policy_arrows = a4.print_policy(vi_policy, width=size, height=size)

    if display_print:
        # display value function:
        print('\n--Value Iteration Values in grid order--')
        a4.matprint(a4.print_value(vi_V, width=size, height=size))
        
        # display policy
        print('\n--Value Iteration Policy Matrix--')
        a4.matprint(vi_policy_arrows)
        
    ## Q-Learning
    print('\n--Q-Learning TimeIt--')
    Q_time = %timeit -o a4.Qlearning(env, qepsilon, lr, qgamma, episodes, initial, decay, report=False)
        
    Q, Q_epochs = a4.Qlearning(env, qepsilon, lr, qgamma, episodes, initial, decay, report)

    maxQ = np.max(Q,axis=1)

    Q_policy = a4.Q_to_policy(Q)

    Q_policy_arrows = a4.print_policy(Q_policy, width=size, height=size)

    if display_print: 
        print('--Q with all options--')
        a4.matprint(Q)
        print('\n--argmax(Q) in grid order--')
        a4.matprint(a4.print_value(maxQ, width=size, height=size))
        print('\n--Q-Learning Policy Matrix--')
        a4.matprint(Q_policy_arrows)
        
    ## Save results to DataFrame
    results = pd.DataFrame({'rH': [rH], 
                        'rG': [rG], 
                        'rF': [rF], 
                        'size': [size], 
                        'p': [p], 
                        'desc': [desc], 
                        'map_name': [map_name],                        
                        'is_slippery': [is_slippery],
                        'epsilon': [epsilon],
                        'gamma': [gamma], 
                        'max_iter': [max_iter], 
                        'qepsilon': [qepsilon], 
                        'lr': [lr], 
                        'qgamma': [qgamma], 
                        'episodes': [episodes], 
                        'initial': [initial],
                        'env_desc': [env_desc],
                        'env_rs': [env_rs],
                        'pi_time': [pi_time.average],
                        'pi_V': [pi_V],
                        'pi_epochs': [pi_epochs],
                        'pi_policy': [pi_policy],
                        'pi_policy_arrows': [pi_policy_arrows],
                        'vi_time': [vi_time.average],
                        'vi_V': [vi_V],
                        'vi_epochs': [vi_epochs],
                        'vi_policy': [vi_policy],
                        'vi_policy_arrows': [vi_policy_arrows],
                        'Q_time': [Q_time.average],
                        'Q': [Q],
                        'Q_epochs': [Q_epochs],
                        'Q_V': [maxQ],
                        'Q_policy': [Q_policy],
                        'Q_policy_arrows': [Q_policy_arrows]})
    
    if display_print: 
        display(results)
        
    ## Save results to disk
    warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
    try:
        dataset = pd.read_hdf('data.h5', key='dataset', mode='a')
    except FileNotFoundError:
        results.to_hdf('data.h5', key='dataset', mode='a')
    else:
        dataset.append(
            other=results, 
            ignore_index=True,
            sort=False
            ).to_hdf(
            path_or_buf='data.h5', 
            key='dataset', 
            mode='a')
        
    if display_print:
        pd.read_hdf('data.h5', key='dataset', mode='a')
    
    
print('Complete!')   

--Board--

SFFF
FHFH
FFFH
HFFG

--Actions for Position to the Left of the Goal--
{0: [(1.0, 13, -0.2, False)],
 1: [(1.0, 14, -0.2, False)],
 2: [(1.0, 15, 1, True)],
 3: [(1.0, 10, -0.2, False)]}

--Reward Values at Each State--
-0.2  -0.2  -0.2  -0.2  
-0.2    -1  -0.2    -1  
-0.2  -0.2  -0.2    -1  
  -1  -0.2  -0.2     1  

--Policy Iteration TimeIt--
1.05 ms ± 18.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Policy iteration converged after  7 epochs

--Policy Iteration Values in grid order--
0.9661  1.4576  2.072  1.4576  
1.4576      -5   2.84      -5  
 2.072    2.84    3.8      -5  
    -5     3.8      5       5  

--Policy Iteration Policy Matrix--
↓  →  ↓  ←  
↓  ←  ↓  ←  
→  ↓  ↓  ←  
←  →  →  ←  

--Value Iteration TimeIt--
6.98 ms ± 69.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Value iteration converged after  91 epochs

--Value Iteration Values in grid order--
0.9661  1.4576  2.072  1.4576  
1.4576      -5   2.84      -5  
 2.072    2.84   

,rH,rG,rF,size,p,desc,map_name,is_slippery,epsilon,gamma,max_iter,qepsilon,lr,qgamma,episodes,initial,env_desc,env_rs,pi_time,pi_V,pi_epochs,pi_policy,pi_policy_arrows,vi_time,vi_V,vi_epochs,vi_policy,vi_policy_arrows,Q_time,Q,Q_epochs,Q_V,Q_policy,Q_policy_arrows
0,-1,1,-0.2,4,0.8,"[SFFF, FHFH, FFFH, HFFG]",4x4,False,1.000000e-08,0.8,10000,0.1,0.8,0.95,10000,1,"[[S, F, F, F], [F, H, F, H], [F, F, F, H], [H,...","[-0.2, -0.2, -0.2, -0.2, -0.2, -1.0, -0.2, -1....",0.001048,"[0.9660800000000008, 1.457600000000001, 2.0720...",7,"[1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0]","[[↓, →, ↓, ←], [↓, ←, ↓, ←], [→, ↓, ↓, ←], [←,...",0.00698,"[0.9660799905143127, 1.4575999905143127, 2.071...",91,"[1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0]","[[↓, →, ↓, ←], [↓, ←, ↓, ←], [→, ↓, ↓, ←], [←,...",0.012021,"[[0.3737963529136319, 0.6039965355670365, 0.55...",43,"[0.6039965355670365, 0.8340003724087158, 1.099...","[1, 2, 1, 2, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0]","[[↓, →, ↓, →], [↓, ←, ↓, ←], [→, ↓, ↓, ←], [←,..."


--Board--

SFFF
FHFH
FFFH
HFFG

--Actions for Position to the Left of the Goal--
{0: [(1.0, 13, -0.2, False)],
 1: [(1.0, 14, -0.2, False)],
 2: [(1.0, 15, 1, True)],
 3: [(1.0, 10, -0.2, False)]}

--Reward Values at Each State--
-0.2  -0.2  -0.2  -0.2  
-0.2    -1  -0.2    -1  
-0.2  -0.2  -0.2    -1  
  -1  -0.2  -0.2     1  

--Policy Iteration TimeIt--
1.03 ms ± 8.53 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Policy iteration converged after  7 epochs

--Policy Iteration Values in grid order--
0.9661  1.4576  2.072  1.4576  
1.4576      -5   2.84      -5  
 2.072    2.84    3.8      -5  
    -5     3.8      5       5  

--Policy Iteration Policy Matrix--
↓  →  ↓  ←  
↓  ←  ↓  ←  
→  ↓  ↓  ←  
←  →  →  ←  

--Value Iteration TimeIt--
7 ms ± 103 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Value iteration converged after  91 epochs

--Value Iteration Values in grid order--
0.9661  1.4576  2.072  1.4576  
1.4576      -5   2.84      -5  
 2.072    2.84    3.8

,rH,rG,rF,size,p,desc,map_name,is_slippery,epsilon,gamma,max_iter,qepsilon,lr,qgamma,episodes,initial,env_desc,env_rs,pi_time,pi_V,pi_epochs,pi_policy,pi_policy_arrows,vi_time,vi_V,vi_epochs,vi_policy,vi_policy_arrows,Q_time,Q,Q_epochs,Q_V,Q_policy,Q_policy_arrows
0,-1,1,-0.2,4,0.8,"[SFFF, FHFH, FFFH, HFFG]",4x4,False,1.000000e-08,0.8,10000,0.1,0.6,0.95,10000,1,"[[S, F, F, F], [F, H, F, H], [F, F, F, H], [H,...","[-0.2, -0.2, -0.2, -0.2, -0.2, -1.0, -0.2, -1....",0.001032,"[0.9660800000000008, 1.457600000000001, 2.0720...",7,"[1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0]","[[↓, →, ↓, ←], [↓, ←, ↓, ←], [→, ↓, ↓, ←], [←,...",0.007002,"[0.9660799905143127, 1.4575999905143127, 2.071...",91,"[1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0]","[[↓, →, ↓, ←], [↓, ←, ↓, ←], [→, ↓, ↓, ←], [←,...",0.015068,"[[0.37379406083697947, 0.6039964852531244, 0.5...",64,"[0.6039964852531244, 0.8443149112725706, 1.101...","[1, 2, 1, 0, 1, 0, 1, 0, 2, 2, 1, 0, 0, 2, 2, 0]","[[↓, →, ↓, ←], [↓, ←, ↓, ←], [→, →, ↓, ←], [←,..."


--Board--

SFFF
FHFH
FFFH
HFFG

--Actions for Position to the Left of the Goal--
{0: [(1.0, 13, -0.2, False)],
 1: [(1.0, 14, -0.2, False)],
 2: [(1.0, 15, 1, True)],
 3: [(1.0, 10, -0.2, False)]}

--Reward Values at Each State--
-0.2  -0.2  -0.2  -0.2  
-0.2    -1  -0.2    -1  
-0.2  -0.2  -0.2    -1  
  -1  -0.2  -0.2     1  

--Policy Iteration TimeIt--
1.15 ms ± 84.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Policy iteration converged after  7 epochs

--Policy Iteration Values in grid order--
0.9661  1.4576  2.072  1.4576  
1.4576      -5   2.84      -5  
 2.072    2.84    3.8      -5  
    -5     3.8      5       5  

--Policy Iteration Policy Matrix--
↓  →  ↓  ←  
↓  ←  ↓  ←  
→  ↓  ↓  ←  
←  →  →  ←  

--Value Iteration TimeIt--
7.47 ms ± 615 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Value iteration converged after  91 epochs

--Value Iteration Values in grid order--
0.9661  1.4576  2.072  1.4576  
1.4576      -5   2.84      -5  
 2.072    2.84    

...

--Board--

SFFFFFHHFFFFFFFF
FFFFHFFHFFFFFFHF
FHFFFFFHFHFHFFFF
HFFHFFFFFFFFFHFH
FHFFFFHHFFFHHFHF
FHFFHFHFFFFHFFFF
FFHFFFFHFFFHHHHF
HFHFFFFFFFFFFFFH
FFFFFFFFFHFFFFHF
FFFFFFHHFHFFFFFH
FFHFFFFFFHFFFHFF
FFFHHFHFFFFFHFFF
FFFFFHFHFFFFHFFF
HHHHHFFFHFFHFFFF
FHFFHFFFFFFFFFFF
FFHFHHFFHFFFFFFG

--Actions for Position to the Left of the Goal--
{0: [(1.0, 253, -0.2, False)],
 1: [(1.0, 254, -0.2, False)],
 2: [(1.0, 255, 1, True)],
 3: [(1.0, 238, -0.2, False)]}

--Reward Values at Each State--
-0.2  -0.2  -0.2  -0.2  -0.2  -0.2    -1    -1  -0.2  -0.2  -0.2  -0.2  -0.2  -0.2  -0.2  -0.2  
-0.2  -0.2  -0.2  -0.2    -1  -0.2  -0.2    -1  -0.2  -0.2  -0.2  -0.2  -0.2  -0.2    -1  -0.2  
-0.2    -1  -0.2  -0.2  -0.2  -0.2  -0.2    -1  -0.2    -1  -0.2    -1  -0.2  -0.2  -0.2  -0.2  
  -1  -0.2  -0.2    -1  -0.2  -0.2  -0.2  -0.2  -0.2  -0.2  -0.2  -0.2  -0.2    -1  -0.2    -1  
-0.2    -1  -0.2  -0.2  -0.2  -0.2    -1    -1  -0.2  -0.2  -0.2    -1    -1  -0.2    -1  -0.2  
-0.2    -1  -0.2  -0.2    -1 

Q-Learning converged after  322 epochs
--Q with all options--
  -2.23081    -2.14987    -2.14987    -2.24225  
  -2.24211     -2.0525     -2.0794    -2.12173  
  -2.06298    -1.99609    -1.99514    -2.04676  
  -2.01677    -1.91385    -1.90045    -1.90548  
  -1.89022    -1.94906    -1.85035    -1.81092  
  -1.78616    -1.79414    -1.94528    -1.77602  
        -1          -1          -1          -1  
        -1          -1          -1          -1  
     -1.36    -1.18106    -1.28956    -1.17663  
  -1.20012    -1.24448    -1.28957    -1.28957  
  -1.29534    -1.26025    -1.34759    -1.28957  
  -1.27795    -1.34707    -1.37599    -1.34657  
  -1.32913    -1.36169    -1.28322    -1.28957  
  -1.34113    -1.28127     -1.2405    -1.28957  
  -1.28843       -1.36    -1.22993    -1.17663  
  -1.28529    -1.19713    -1.28957    -1.28957  
  -2.14894     -2.0525     -2.0525    -2.21985  
   -2.1388       -1.95    -1.99312     -2.0048  
  -1.92241    -1.91024    -1.92624     -2.0313  
  -1.98

↑  →  →  →  ←  ←  ↓  ←  →  →  ↓  ←  ←  →  ↓  ↓  
←  ←  ←  ←  ←  →  ↑  ←  ←  ↓  ↓  ←  ↓  ←  →  ↓  
←  ←  ←  ←  ←  ↑  ↓  ←  →  →  →  →  ↓  ↓  ↓  ↓  
←  ←  ←  ←  ←  ←  ↑  ↓  ←  ↓  ←  →  →  →  →  ←  


,rH,rG,rF,size,p,desc,map_name,is_slippery,epsilon,gamma,max_iter,qepsilon,lr,qgamma,episodes,initial,env_desc,env_rs,pi_time,pi_V,pi_epochs,pi_policy,pi_policy_arrows,vi_time,vi_V,vi_epochs,vi_policy,vi_policy_arrows,Q_time,Q,Q_epochs,Q_V,Q_policy,Q_policy_arrows
0,-1,1,-0.2,16,0.8,"[SFFFFFHHFFFFFFFF, FFFFHFFHFFFFFFHF, FHFFFFFHF...",16x16,False,1.000000e-08,0.2,10000,0.1,0.8,0.95,10000,1,"[[S, F, F, F, F, F, H, H, F, F, F, F, F, F, F,...","[-0.2, -0.2, -0.2, -0.2, -0.2, -0.2, -1.0, -1....",0.067487,"[-0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -1....",12,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, ...","[[←, ←, ←, ←, ←, ←, ←, ←, ↓, ↓, ↓, ←, ←, ←, ←,...",0.023044,"[-0.24999999995904001, -0.24999999995904001, -...",15,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[[←, ←, ←, ←, ←, ←, ←, ←, ↓, ←, ←, ←, ←, ←, ←,...",0.352674,"[[-2.2308136923961635, -2.1498745270375994, -2...",322,"[-2.1498745270375994, -2.0524999426173984, -1....","[1, 1, 2, 2, 3, 3, 0, 0, 3, 0, 1, 0, 2, 2, 3, ...","[[↓, ↓, →, →, ↑, ↑, ←, ←, ↑, ←, ↓, ←, →, →, ↑,..."


Complete!


# Notes

Default rewards in OpenAI gym Frozen-Lake-v0 are 1 for the G(oal) and 0 for everything else.

Maps are drawn according to the following logic

```
if desc and map_name are None, 
   then a default random map is drawn with 8
        using frozen_lake.generate_random_map(size=8, p=0.8)
elif desc is None and a map_name is given
   then a map_name is either '4x4' or '8x8'
        and is drawn from the dict MAPS in frozen_lake.py
elif desc is given
   then it must be in the form of a list with 
```

Default action probabilities are 1/3 chosen action, 1/3 each for right angles to chosen action, and 0 for reverse of chosen action. This is set with `is_slippery=True`. If `is_slippery=False`, then P=1 for chosen action and 0 for all other actions.

|ACTION|Value|Symbol|
|------|-----|------|
|LEFT  | 0   | ←    |
|DOWN  | 1   | ↓    |
|RIGHT | 2   | →    |
|UP    | 3   | ↑    |

# Sources

- Environment: <https://gym.openai.com/envs/FrozenLake-v0/>
- Code: <https://github.com/Twice22/HandsOnRL>
- Tutorial: <https://twice22.github.io/>